In [1]:
import numpy as np
import string

rows = []
with open('day_12_input.txt') as file:
    while line:= file.readline().strip():
        row = [c for c in line]
        rows.append(row)

In [2]:
grid = np.array(rows)
end_state = np.argwhere(grid == 'E').squeeze()
start_state = np.argwhere(grid == 'S').squeeze()
q_values = np.zeros((*grid.shape, 4)) * 10
height = {string.ascii_lowercase[i]:i for i in range(len(string.ascii_lowercase))}
height['S'] = height['a']
height['E'] = height['z']

In [3]:
# for n in range(grid.shape[0] * grid.shape[1]):
#     steps = 0
#     print(n)
#     for i in range(grid.shape[0]):
#         for j in range(grid.shape[1]):
#             state = np.array([i, j])
#             if np.all(state == end_state):
#                 continue
#             for action in range(4):
#                 reward = -1
#                 if action == 0:
#                     new_state = state - [1, 0]
#                     if new_state[0] < 0 or height[grid[tuple(new_state)]] - height[grid[tuple(state)]] > 1:
#                         reward -= 10 * grid.shape[0] * grid.shape[1]
#                         new_state = state
#                 elif action == 1:
#                     new_state = state + [0, 1]
#                     if new_state[1] == grid.shape[1] or height[grid[tuple(new_state)]] - height[grid[tuple(state)]] > 1:
#                         reward -= 10 * grid.shape[0] * grid.shape[1]
#                         new_state = state
#                 elif action == 2:
#                     new_state = state + [1, 0]
#                     if new_state[0] == grid.shape[0] or height[grid[tuple(new_state)]] - height[grid[tuple(state)]] > 1:
#                         reward -= 10 * grid.shape[0] * grid.shape[1]
#                         new_state = state
#                 elif action == 3:
#                     new_state = state - [0, 1]
#                     if new_state[1] < 0 or height[grid[tuple(new_state)]] - height[grid[tuple(state)]] > 1:
#                         reward -= 10 * grid.shape[0] * grid.shape[1]
#                         new_state = state
#                 if np.all(new_state == end_state):
#                     reward += 5 * grid.shape[0] * grid.shape[1]
#                 q_values[tuple(list(state) + [action])] += alpha * (
#                     reward + q_values[tuple(new_state)].max() - q_values[tuple(list(state) + [action])])
#                 steps += 1
#     if np.any(q_values[tuple(start_state)] > 0):
#         break

In [6]:
# q-learning version:
from tqdm import tqdm
alpha = 1
epsilon = 0

for i in tqdm(range(100000)):
    state = start_state
    steps = 0
    while not np.all(state == end_state) and steps < 1000:
        steps += 1
        if np.random.random() < epsilon:
            action = np.random.choice(4)
        else:
            arg_maxx = np.argwhere(q_values[tuple(state)] == q_values[tuple(state)].max())
            if len(arg_maxx) == 1:
                action = arg_maxx[0, 0]
            else:
                action = np.random.choice(arg_maxx.squeeze())
        reward = -1
        if action == 0:
            new_state = state - [1, 0]
            if new_state[0] < 0 or height[grid[tuple(new_state)]] - height[grid[tuple(state)]] > 1:
                reward -= 100000
                new_state = state
        elif action == 1:
            new_state = state + [0, 1]
            if new_state[1] == grid.shape[1] or height[grid[tuple(new_state)]] - height[grid[tuple(state)]] > 1:
                reward -= 100000
                new_state = state
        elif action == 2:
            new_state = state + [1, 0]
            if new_state[0] == grid.shape[0] or height[grid[tuple(new_state)]] - height[grid[tuple(state)]] > 1:
                reward -= 100000
                new_state = state
        elif action == 3:
            new_state = state - [0, 1]
            if new_state[1] < 0 or height[grid[tuple(new_state)]] - height[grid[tuple(state)]] > 1:
                reward -= 100000
                new_state = state
        if np.all(new_state == end_state):
            reward += 10000
        q_values[tuple(list(state) + [action])] += alpha * (
            reward + q_values[tuple(new_state)].max() - q_values[tuple(list(state) + [action])])
        state = new_state
    if np.any(q_values[tuple(list(start_state))] > 0):
        break

  8%|██████▎                                                                    | 8366/100000 [13:19<2:25:52, 10.47it/s]


In [7]:
# Task 1

state = start_state
steps = 0
while not np.all(state == end_state):
    action = q_values[tuple(state)].argmax()
    if action == 0:
        state = state - [1, 0]
    elif action == 1:
        state = state + [0, 1]
    elif action == 2:
        state = state + [1, 0]
    elif action == 3:
        state = state - [0, 1]
    steps += 1
steps

462

In [8]:
# Task 2
avail_values = q_values.max(axis=2) * (np.vectorize(height.get)(grid) == 0)
i = avail_values.max(axis=1).argmax()
j = avail_values.max(axis=0).argmax()
state = np.array([i, j])
steps = 0
while not np.all(state == end_state):
    action = q_values[tuple(state)].argmax()
    if action == 0:
        state = state - [1, 0]
    elif action == 1:
        state = state + [0, 1]
    elif action == 2:
        state = state + [1, 0]
    elif action == 3:
        state = state - [0, 1]
    steps += 1
steps

460